In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
from pathlib import Path

In [75]:
import joblib

In [4]:
import re

In [41]:
from sklearn.model_selection import train_test_split

In [48]:
from sklearn.compose import ColumnTransformer

In [49]:
from sklearn.pipeline import Pipeline

In [51]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [52]:
from sklearn.impute import SimpleImputer

In [5]:
RANDOM_STATE = 42

In [6]:
BASE = Path('.')

In [7]:
RAW_PATH = BASE / 'data' / 'raw' / 'online_shoppers_intention.csv'

In [8]:
df_raw = pd.read_csv(RAW_PATH)

In [9]:
df = df_raw.copy()

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12330 entries, 0 to 12329
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Administrative           12330 non-null  int64  
 1   Administrative_Duration  12330 non-null  float64
 2   Informational            12330 non-null  int64  
 3   Informational_Duration   12330 non-null  float64
 4   ProductRelated           12330 non-null  int64  
 5   ProductRelated_Duration  12330 non-null  float64
 6   BounceRates              12330 non-null  float64
 7   ExitRates                12330 non-null  float64
 8   PageValues               12330 non-null  float64
 9   SpecialDay               12330 non-null  float64
 10  Month                    12330 non-null  object 
 11  OperatingSystems         12330 non-null  int64  
 12  Browser                  12330 non-null  int64  
 13  Region                   12330 non-null  int64  
 14  TrafficType           

In [11]:
df.describe()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,OperatingSystems,Browser,Region,TrafficType
count,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000
mean,2.315166,80.818611,0.503569,34.472398,31.731468,1194.746220,0.022191,0.043073,5.889258,0.061427,2.124006,2.357097,3.147364,4.069586
std,3.321784,176.779107,1.270156,140.749294,44.475503,1913.669288,0.048488,0.048597,18.568437,0.198917,0.911325,1.717277,2.401591,4.025169
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,7.000000,184.137500,0.000000,0.014286,0.000000,0.000000,2.000000,2.000000,1.000000,2.000000
50%,1.000000,7.500000,0.000000,0.000000,18.000000,598.936905,0.003112,0.025156,0.000000,0.000000,2.000000,2.000000,3.000000,2.000000
75%,4.000000,93.256250,0.000000,0.000000,38.000000,1464.157214,0.016813,0.050000,0.000000,0.000000,3.000000,2.000000,4.000000,4.000000
max,27.000000,3398.750000,24.000000,2549.375000,705.000000,63973.522230,0.200000,0.200000,361.763742,1.000000,8.000000,13.000000,9.000000,20.000000


In [12]:
def to_snake(s: str) -> str:
    s = re.sub(r"[^0-9a-zA-Z]+", "_", s)
    s = re.sub(r"([a-z0-9])([A-Z])", r"\1_\2", s)
    return s.strip("_").lower()

In [13]:
df.columns = [to_snake(c) for c in df.columns]

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12330 entries, 0 to 12329
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   administrative            12330 non-null  int64  
 1   administrative_duration   12330 non-null  float64
 2   informational             12330 non-null  int64  
 3   informational_duration    12330 non-null  float64
 4   product_related           12330 non-null  int64  
 5   product_related_duration  12330 non-null  float64
 6   bounce_rates              12330 non-null  float64
 7   exit_rates                12330 non-null  float64
 8   page_values               12330 non-null  float64
 9   special_day               12330 non-null  float64
 10  month                     12330 non-null  object 
 11  operating_systems         12330 non-null  int64  
 12  browser                   12330 non-null  int64  
 13  region                    12330 non-null  int64  
 14  traffi

In [15]:
for col in ['weekend','revenue']:
    if col in df.columns:
        df[col] = df[col].astype(int)

In [24]:
dup_count = df.duplicated().sum()

In [25]:
dup_count

np.int64(0)

In [26]:
if dup_count > 0:
    df = df.drop_duplicates().reset_index(drop=True)

In [27]:
duration_cols = [c for c in df.columns if 'duration' in c]

In [28]:
(df[duration_cols] < 0).sum().sort_values(ascending=False).head(10)

administrative_duration     0
informational_duration      0
product_related_duration    0
dtype: int64

In [29]:
##eğer olsaydı
## for c in duration_cols:
    ##df.loc[df[c] < 0,c] = np.nan

In [30]:
rate_cols = [c for c in df.columns if 'rate' in c]

In [31]:
for c in rate_cols:
    bad = ((df[c] < 0) | (df[c] > 1)).sum()
    print(c,bad)

bounce_rates 0
exit_rates 0


In [32]:
##eğer olsaydı
##for c in rate_cols:
    ## df.loc[(df[c] < 0 ) | (df[c] > 1),c] = np.nan

In [33]:
missing = df.isna().mean().sort_values(ascending=False)

In [36]:
missing[missing > 0]

Series([], dtype: float64)

In [37]:
cat_cols = df.select_dtypes(include = ['object']).columns.tolist()

In [38]:
for c in cat_cols:
    df[c] = df[c].astype(str).str.strip()
    df.loc[df[c].isin(['','nan','None']),c] = np.nan

In [39]:
if 'month' in df.columns:
    df['month'] = df['month'].str.title()

In [40]:
df['revenue'].value_counts(dropna=False)

revenue
0    10297
1     1908
Name: count, dtype: int64

In [42]:
target = 'revenue'

In [43]:
X = df.drop(columns = [target])

In [44]:
y = df[target]

In [45]:
X_train, X_temp, y_train,y_temp = train_test_split(
    X,y,test_size = 0.30,random_state = RANDOM_STATE, stratify = y
)

In [46]:
X_valid, X_test, y_valid, y_test = train_test_split(
    X_temp,y_temp, test_size = 0.30, random_state = RANDOM_STATE, stratify = y_temp
)

In [47]:
X_train.shape, X_valid.shape, X_test.shape

((8543, 17), (2563, 17), (1099, 17))

In [53]:
num_cols = X_train.select_dtypes(include = [np.number]).columns.tolist()

In [54]:
cat_cols = [c for c in X_train.columns if c not in num_cols]

In [55]:
numeric_pipe = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy = 'median')),
    ('scaler', StandardScaler())
])

In [60]:
categorical_pipe = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy = 'most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

In [64]:
preprocess = ColumnTransformer(
    transformers = [
        ('num', numeric_pipe,num_cols),
        ('cat', categorical_pipe,cat_cols)
    ],
    remainder = 'drop'
)

In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12205 entries, 0 to 12204
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   administrative            12205 non-null  int64  
 1   administrative_duration   12205 non-null  float64
 2   informational             12205 non-null  int64  
 3   informational_duration    12205 non-null  float64
 4   product_related           12205 non-null  int64  
 5   product_related_duration  12205 non-null  float64
 6   bounce_rates              12205 non-null  float64
 7   exit_rates                12205 non-null  float64
 8   page_values               12205 non-null  float64
 9   special_day               12205 non-null  float64
 10  month                     12205 non-null  object 
 11  operating_systems         12205 non-null  int64  
 12  browser                   12205 non-null  int64  
 13  region                    12205 non-null  int64  
 14  traffi

In [66]:
preprocess.fit(X_train)

,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers contains sparse matrices,these will be stacked as a sparse matrix if the overall density islower than this value. Use ``sparse_threshold=0`` to always returndense. When the transformed output consists of all dense data, thestacked result will be dense, and this keyword will be ignored.",0.3
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.",None
,"transformer_weights transformer_weights: dict, default=NoneMultiplicative weights for features per transformer. The output of thetransformer is multiplied by these weights. Keys are transformer names,values the weights.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each transformer will beprinted as it is completed.",False
,"verbose_feature_names_out verbose_feature_names_out: bool, str or Callable[[str, str], str], default=True- If True, :meth:`ColumnTransformer.get_feature_names_out` will prefix all feature names with the name of the transformer that generated that feature. It is equivalent to setting `verbose_feature_names_out=""{transformer_name}__{feature_name}""`.- If False, :meth:`ColumnTransformer.get_feature_names_out` will not prefix any feature names and will error if feature names are not unique.- If ``Callable[[str, str], str]``, :meth:`ColumnTransformer.get_feature_names_out` will rename all the features using the name of the transformer. The first argument of the callable is the transformer name and the second argument is the feature name. The returned string will be the new feature name.- If ``str``, it must be a string ready for formatting. The given string will be formatted using two field names: ``transformer_name`` and ``feature_name``. e.g. `

In [68]:
X_train_p = preprocess.transform(X_train)

In [69]:
X_valid_p = preprocess.transform(X_valid)

In [70]:
X_test_p = preprocess.transform(X_test)

In [72]:
X_train_p.shape, X_valid_p.shape, X_test_p.shape

((8543, 28), (2563, 28), (1099, 28))

In [76]:
(Path('artifacts')).mkdir(parents=True,exist_ok = True)
joblib.dump(preprocess, "artifacts/preprocess_pipeline.joblib")

['artifacts/preprocess_pipeline.joblib']

In [77]:
(Path('data/processed')).mkdir(parents=True, exist_ok=True)
df.to_csv("data/processed/shoppers_clean.csv", index=False)

In [79]:
(Path('data/splits')).mkdir(parents = True, exist_ok = True)
train_df = X_train.copy(); train_df[target] = y_train.values
valid_df = X_valid.copy(); valid_df[target] = y_valid.values
test_df = X_test.copy(); test_df[target] = y_test.values
train_df.to_csv("data/splits/train.csv", index=False)
valid_df.to_csv("data/splits/valid.csv", index=False)
test_df.to_csv("data/splits/test.csv", index=False)